# [CS303]Practice 6 - BTS & Practice 7 - Improving BTS
In Practice 6 and 7, we will apply the BTS algorithm and the Improving BTS to solve the N-Queens problem.

In the following code, only two of the function usages are essential. One is `is_valid(self, state)`, which checks if the current state is legal; the other is `is_satisfy(self, state)`, which is to checks if the current board meets the win condition. 
The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.

In test block, for `Solver`,  `n` indicates the size of the test while `method` indicates which bts function will be used(`bts` or `improving_bts`); for method `solve`, `render` indicates whether to use a graphical interface representation.

## Practice 6 - BTS
You are required to implement complete the function `bts()`.
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. You may add any function you want. 
**Remember to test the case where N=6**.

📌 **DDL: Nov.9th**

## Practice 7 - Improving BTS
You are required to complete the function `improving_bts()`.
For `improving_bts()`, you can use one or more of the following strategies: Minimum Remaining Value, Least constraining value, and Forward checking. You should have a good performance (within 2s) **when N = 16 without GUI**. 

📌 **DDL: Nov.16th**

## Evaluation Criteria

Your submission will be assessed on:

* Clear understanding of every line of your code (no blind copy-paste).
* Successful compilation of the program.
* Correctness of the program logic.
* Reasonable efficiency of the solution.

### Grading Policy

* **On-time submission:** 60–100 points
* **Late submission:** 0 points


In [3]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.N = n

    def is_valid(self, state):
        """
        check whether the state satisfies the condition or not.
        : param state: list of points (in (row id, col id) tuple form)
        : return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.N

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.N - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.N - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.N - i - 1, self.N - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.N, self.N], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.N + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.N + 1))

In [4]:
class Solver:
    def __init__(self, n, method):
        self.N = n
        self.problem = Problem(n)
        self.method = method

    def solve(self, render=True):
        if render:
            import pygame
            w, h = 90 * self.N + 10, 90 * self.N + 10
            screen = pygame.display.set_mode((w, h))
            screen.fill('white')
            action_generator = self.method(self.problem)
            clk = pygame.time.Clock()
            queen_img = pygame.image.load('./queen.png')
            while True:
                for event in pygame.event.get():
                    if event == pygame.QUIT:
                        exit()
                try:
                    actions = next(action_generator)
                    screen.fill('white')
                    for i in range(self.N + 1):
                        pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                        pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                    for action in actions:
                        i, j = action
                        screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                    pygame.display.flip()
                    time.sleep(3)
                except StopIteration:
                    pass
                clk.tick(5)
            pass
        else:
            start_time = time.time()
            for actions in self.method(self.problem):
                pass
            self.problem.print_state(actions)
            print(time.time() - start_time)

# BTS: Backtracking search

In [16]:
def bts(problem):
    action_stack = [] # list of points (row id, col id)
    row = 0  
    col = 0

    while not problem.is_satisfy(action_stack): # 循环直到找到解
        # 回溯：移除上一个皇后
        if row >= problem.N:
            last_row, last_col = action_stack.pop()
            row = last_row
            col = last_col + 1  # 上一个皇后向下一列搜索
            yield action_stack
            continue
        if col >= problem.N:
            last_row, last_col = action_stack.pop()
            row = last_row
            col = last_col + 1 # 上一个皇后向下一列搜索
            yield action_stack
            continue
        
        action_stack.append((row, col)) # 尝试在 (row, col) 放置皇后
        
        if problem.is_valid(action_stack):
            yield action_stack # 如果合法，向下一行从头继续搜索
            row += 1
            col = 0
        else:
            action_stack.pop() # 如果不合法，移除刚放置的皇后，向下一列搜索
            col += 1
    
    yield action_stack


# Improving BTS 
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

## Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
## Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
## Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [ ]:
def improving_bts(problem):
    N = problem.N
    action_stack = []          # list[(row, col)]
    assigned_rows = set()      # 已放置的行
    used_cols = set()          # 已占用列
    used_d1 = set()            # 主对角线 r - c
    used_d2 = set()            # 次对角线 r + c

    def allowed_cols(row):
        # 该行在当前部分解下的可用列（前向检查）
        res = []
        for c in range(N):
            if (c in used_cols) or ((row - c) in used_d1) or ((row + c) in used_d2):
                continue
            res.append(c)
        return res

    # 栈帧： [row, candidates(list), idx(int)]
    frames = []

    # 初始状态也 yield，便于 GUI 渲染
    yield action_stack

    while True:
        # 成功：放满 N 个皇后
        if len(action_stack) == N:
            yield action_stack
            return

        # 选未赋值变量中取值域最小的行（MRV）
        unassigned = [r for r in range(N) if r not in assigned_rows]
        best_row, best_cands, min_size = None, None, None
        for r in unassigned:
            cands = allowed_cols(r)
            if min_size is None or len(cands) < min_size:
                best_row, best_cands, min_size = r, cands, len(cands)
            if min_size == 0: # FC
                break

        if not best_cands:
            # 无可用值：回溯
            while frames:
                row, cands, idx = frames[-1]
                # 撤销当前放置
                cur_col = cands[idx]
                assert action_stack and action_stack[-1] == (row, cur_col)
                action_stack.pop()
                assigned_rows.remove(row)
                used_cols.remove(cur_col)
                used_d1.remove(row - cur_col)
                used_d2.remove(row + cur_col)

                # 换该行的下一个候选
                idx += 1
                if idx < len(cands):
                    frames[-1][2] = idx
                    col = cands[idx]
                    action_stack.append((row, col))
                    assigned_rows.add(row)
                    used_cols.add(col)
                    used_d1.add(row - col)
                    used_d2.add(row + col)
                    yield action_stack
                    break
                else:
                    # 该行候选也耗尽，继续向上回溯
                    frames.pop()
                    yield action_stack
            else:
                # 根也回溯完：无解
                yield action_stack
                return
            continue
        
        # LCV：对 best_row 的候选列按“对其它未赋值行的约束数”升序排序
        remaining_rows = [r for r in unassigned if r != best_row]
        # 预先计算其它行当前的可用列集合，加速评估
        avail = {r: set(allowed_cols(r)) for r in remaining_rows}

        def lcv_cost(col):
            cost = 0
            r = best_row
            for r2 in remaining_rows:
                d = r2 - r
                # 放置 (r,col) 会使 r2 行中以下列不可用：同列、两条对角线
                forbid = set()
                c_same = col
                c_d1 = col + d        # r-c 不变 -> c' = c + (r2 - r)
                c_d2 = col - d        # r+c 不变 -> c' = c - (r2 - r)
                if 0 <= c_same < N:  forbid.add(c_same)
                if 0 <= c_d1  < N:   forbid.add(c_d1)
                if 0 <= c_d2  < N:   forbid.add(c_d2)
                # 仅统计那些当前仍是可选值的列
                cost += sum(1 for x in forbid if x in avail[r2])
            return cost

        # 将候选按 LCV 代价排序（越小越优）
        cands = sorted(best_cands, key=lcv_cost)

        # 有可用行：压栈并放置它的第一个候选列
        row = best_row
        cands = best_cands  # 可按需应用 LCV 对 cands 排序
        frames.append([row, cands, 0])
        col = cands[0]
        action_stack.append((row, col))
        assigned_rows.add(row)
        used_cols.add(col)
        used_d1.add(row - col)
        used_d2.add(row + col)
        yield action_stack

## test block

In [19]:
Solver(n=6, method=bts).solve(render=False)

_____________
|·|Q|·|·|·|·|
|·|·|·|Q|·|·|
|·|·|·|·|·|Q|
|Q|·|·|·|·|·|
|·|·|Q|·|·|·|
|·|·|·|·|Q|·|
-------------
0.014299154281616211


In [14]:
Solver(n=16, method=improving_bts).solve(render=False)

_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
---------------------------------
0.0033380985260009766
